<a href="https://colab.research.google.com/github/AmandinChyba/Project1-Fields-2022/blob/main/Walk_on_Boundary_and_BIEM/WalkOnBoundary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax
from jax import lax
from jax import random

import jax.numpy as jnp
import numpy as np

import time

import matplotlib.pyplot as plt

from functools import partial

In [2]:
def unit_circle_boundary(x, y):
  return jnp.sqrt(jnp.power(x, 2) + jnp.power(y, 2)) - 1

In [11]:
def getMiddle(p0, p1):
  return jnp.array([(p1[0]-p0[0])/2 + p0[0], (p1[1]-p0[1])/2 + p0[1]])

In [41]:
def binaryStep(epoch, bounds):
  mid = getMiddle(bounds[0],bounds[1])
  dist = jnp.where(distanceF(mid[0],mid[1]) > 0, 1, 0)
  return jnp.array([mid*(1-dist) + bounds[0]*dist, mid*dist + bounds[1]*(1-dist)])
  # can't use this code because of the if-statement
  '''
  mid = getMiddle(bounds[0],bounds[1])
  if distanceF(mid[0],mid[1]) > 0:
    nextBounds = jnp.array([bounds[0],mid])
  else:
    nextBounds = jnp.array([mid,bounds[1]])
  return nextBounds
  '''

In [20]:
def binaryRootSearch(point, angle, distanceF, epochs, forward):
  epsilon = 0.01
  diameter = 2 + epsilon

  lower = point
  # optimize this
  if forward:
    upper = jnp.array([point[0] + diameter * jnp.cos(angle), point[1] + diameter * jnp.sin(angle)])
  else:
    upper = jnp.array([point[0] - diameter * jnp.cos(angle), point[1] - diameter * jnp.sin(angle)])
  
  bounds = jnp.array([lower,upper])
  bounds = lax.fori_loop(0, epochs, binaryStep, bounds)

  root = getMiddle(bounds[0],bounds[1])
  return root

In [12]:
def getDistance(p0, p1):
  return jnp.array([jnp.power(p1[0]-p0[0],2) + jnp.power(p1[1]-p0[1],2)]) # distance squared

In [7]:
def wob(g, distanceF, x0, y0, t, epochs, angles):
  runningEst = 0
  runningSign = 1

  pInit = jnp.array([x0,y0])
  pNext = binaryRootSearch(pInit, angles[0], distanceF, epochs, True) 

  for angle in angles[1:]:
    runningEst += runningSign * g(pNext)
    runningSign *= -1

    p0 = binaryRootSearch(pNext, angle, distanceF, epochs, False)
    p1 = binaryRootSearch(pNext, angle, distanceF, epochs, True)
    pNext = jnp.where(getDistance(pNext,p0) - getDistance(pNext,p1) > 0, p0, p1)

  runningEst *= 2
  runningEst += runningSign * g(pNext)

  return runningEst

In [40]:
%%time
t = 10 # number of jumps on the boundary
epochs = 25 # binary search time steps
batches = 1000000

g = lambda p : p[1] # p is a point s.t. p[0]=x and p[1]=y
distanceF = unit_circle_boundary

x0 = np.array([0]*batches)
y0 = np.array([-0.3]*batches)

key = random.PRNGKey(0)
angles = random.uniform(key, shape=(t, batches)) * 2 * jnp.pi

ans = wob(g, distanceF, x0, y0, t, epochs, angles)

mean = jnp.mean(ans)
print(mean)

-0.30611658
CPU times: user 8.51 s, sys: 371 ms, total: 8.88 s
Wall time: 5.78 s
